# Performing Attribute join and Spatial Joins

In [2]:
import geopandas as gpd
import geodatasets

In [17]:
chicago = gpd.read_file(geodatasets.get_path("geoda.chicago_commpop"))
groceries = gpd.read_file(geodatasets.get_path("geoda.groceries"))

In [4]:
chicago.head()

,community,NID,POP2010,POP2000,POPCH,POPPERCH,popplus,popneg,geometry
0,DOUGLAS,35,18238,26470,-8232,-31.099358,0,1,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ..."
1,OAKLAND,36,5918,6110,-192,-3.142390,0,1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ..."
2,FULLER PARK,37,2876,3420,-544,-15.906433,0,1,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ..."
3,GRAND BOULEVARD,38,21929,28006,-6077,-21.698922,0,1,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ..."
4,KENWOOD,39,17841,18363,-522,-2.842673,0,1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ..."


In [4]:
groceries.head()

,OBJECTID,Ycoord,Xcoord,Status,Address,Chain,Category,geometry
0,16,41.973266,-87.657073,OPEN,"1051 W ARGYLE ST, CHICAGO, IL. 60640",VIET HOA PLAZA,None,MULTIPOINT ((1168268.672 1933554.350))
1,18,41.696367,-87.681315,OPEN,"10800 S WESTERN AVE, CHICAGO, IL. 60643-3226",COUNTY FAIR FOODS,None,MULTIPOINT ((1162302.618 1832900.224))
2,22,41.868634,-87.638638,OPEN,"1101 S CANAL ST, CHICAGO, IL. 60607-4932",WHOLE FOODS MARKET,None,MULTIPOINT ((1173317.042 1895425.426))
3,23,41.877590,-87.654953,OPEN,"1101 W JACKSON BLVD, CHICAGO, IL. 60607-2905",TARGET/SUPER,new,MULTIPOINT ((1168996.475 1898801.406))
4,27,41.737696,-87.625795,OPEN,"112 W 87TH ST, CHICAGO, IL. 60620-1318",FOOD 4 LESS,None,MULTIPOINT ((1176991.989 1847262.423))


## For Attribute join:

In [5]:
chicago_shapes = chicago[['geometry', 'NID']] #this is a GeoDataFrame
chicago_names = chicago[['community', 'NID']] #this is a DataFrame

In [6]:
chicago_shapes.head()

,geometry,NID
0,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35
1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36
2,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ...",37
3,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ...",38
4,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39


In [7]:
chicago_names.head()

,community,NID
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39


## For Spatial Join:

In [8]:
chicago = chicago[['geometry', 'community']].to_crs(groceries.crs)

In [9]:
chicago.head()

,geometry,community
0,"MULTIPOLYGON (((1181573.250 1886828.039, 11815...",DOUGLAS
1,"MULTIPOLYGON (((1186289.356 1876750.733, 11862...",OAKLAND
2,"MULTIPOLYGON (((1176344.998 1871187.546, 11763...",FULLER PARK
3,"MULTIPOLYGON (((1182322.043 1876674.730, 11823...",GRAND BOULEVARD
4,"MULTIPOLYGON (((1186289.356 1876750.733, 11862...",KENWOOD


## 1. Attribute Join: using the merge() method.
### When performing attribute join on GeoDataFrames, the GeoDataFrame should be on the left side. We are joining a GeoDataFrame to a DataFrame

In [10]:
chicago_shapes.head()

,geometry,NID
0,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35
1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36
2,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ...",37
3,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ...",38
4,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39


In [11]:
chicago_names.head()

,community,NID
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39


## The actual merging:

In [12]:
chicago_shapes = chicago_shapes.merge(chicago_names, on = 'NID')

In [13]:
chicago_shapes.head()

,geometry,NID,community
0,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35,DOUGLAS
1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36,OAKLAND
2,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ...",37,FULLER PARK
3,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ...",38,GRAND BOULEVARD
4,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39,KENWOOD


#

#

#

# 2. Performing a Spatial Join: using sjoin()

In [14]:
groceries_with_community = groceries.sjoin(chicago, how = 'inner', predicate='intersects')

In [15]:
groceries_with_community.head()

,OBJECTID,Ycoord,Xcoord,Status,Address,Chain,Category,geometry,index_right,community
0,16,41.973266,-87.657073,OPEN,"1051 W ARGYLE ST, CHICAGO, IL. 60640",VIET HOA PLAZA,None,MULTIPOINT ((1168268.672 1933554.350)),30,UPTOWN
1,18,41.696367,-87.681315,OPEN,"10800 S WESTERN AVE, CHICAGO, IL. 60643-3226",COUNTY FAIR FOODS,None,MULTIPOINT ((1162302.618 1832900.224)),73,MORGAN PARK
2,22,41.868634,-87.638638,OPEN,"1101 S CANAL ST, CHICAGO, IL. 60607-4932",WHOLE FOODS MARKET,None,MULTIPOINT ((1173317.042 1895425.426)),28,NEAR WEST SIDE
3,23,41.877590,-87.654953,OPEN,"1101 W JACKSON BLVD, CHICAGO, IL. 60607-2905",TARGET/SUPER,new,MULTIPOINT ((1168996.475 1898801.406)),28,NEAR WEST SIDE
4,27,41.737696,-87.625795,OPEN,"112 W 87TH ST, CHICAGO, IL. 60620-1318",FOOD 4 LESS,None,MULTIPOINT ((1176991.989 1847262.423)),39,CHATHAM


In [18]:
new_chicago = chicago.loc[:, ['geometry', 'NID']].copy()

In [19]:
new_chicago.head()

,geometry,NID
0,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35
1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36
2,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ...",37
3,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ...",38
4,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39
